In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [7]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [9]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 8-29-24.csv')

#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()

# Seperate dataset as response variable and feature variables
data = data.drop(['Substrate','Hirshfeld Heavy Atom Charge','CM5 Charge','Hirshfeld Carbon Charge','Hirshfeld Hydrogen Charge',
              'ESP Heavy Atom Charge','ESP Carbon Charge','ESP Hydrogen Charge','NPA Carbon Charge','NPA Hydrogen Charge',
              'MBS Heavy Atom Charge','MBS Carbon Charge','MBS Hydrogen Charge','Mulliken Heavy Charge','Mulliken Carbon Charge',
              'Mulliken Hydrogen Charge','Product'], axis = 1)

In [10]:
Results_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

maxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
  
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_X = training_X.drop('grouped', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_X = test_X.drop('grouped', axis = 1)
    test_y = test_data['Product_Ratio']
 
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-23] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
              0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
              0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55]:
             
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred = rfc.predict(test_X)
        Results_df = classificationMetrics(Results_df, test_y, pred)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = Results_df.groupby('y')['acc'].mean()
        acc_std = Results_df.groupby('y')['acc'].std()
        precision_0_mean = Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = Results_df.groupby('y')['precision 0'].std()
        precision_1_mean = Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = Results_df.groupby('y')['recall 0'].std()
        recall_1_mean = Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = Results_df.groupby('y')['F1 1'].std()
        true_neg_mean = Results_df.groupby('y')['True Neg'].mean()
        true_neg_std = Results_df.groupby('y')['True Neg'].std()
        false_pos_mean = Results_df.groupby('y')['False Pos'].mean()
        false_pos_std = Results_df.groupby('y')['False Pos'].std()        
        false_neg_mean = Results_df.groupby('y')['False Neg'].mean()
        false_neg_std = Results_df.groupby('y')['False Neg'].std()      
        true_pos_mean = Results_df.groupby('y')['True Pos'].mean() 
        true_pos_std = Results_df.groupby('y')['True Pos'].std()   
        

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)
    
        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                 

        maxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        maxacc_copy  = maxacc.copy()
        maxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)
        #print(x,y)
    test_index = pd.DataFrame(test_data.index.values)
    test_index_total = pd.concat([test_index_total, test_index],axis = 1)
    test_y = test_y.rename(x)
    prod = prod.append(test_y)
    maxacc_comb = maxacc_comb.append(maxacc_copy)  

total_results = val_tot.T
 
maxacc_comb.to_csv("JchemExp.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("JchemExp.csv", mode="a")

In [11]:
maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.30,0.745024,0.005826,0.871369,0.004842,0.370221,0.012350,0.804094,0.003082,0.492500,0.020582,0.836377,0.003590,0.422682,0.015522,137.500000,0.527046,33.500000,0.527046,20.300000,0.823273,19.700000,0.823273,1
0.25,0.749867,0.008683,0.861298,0.010350,0.362743,0.014399,0.824720,0.022522,0.428750,0.064520,0.842322,0.007921,0.391356,0.032539,141.450000,4.273234,30.050000,3.776311,22.850000,2.580800,17.150000,2.580800,2
0.25,0.749606,0.007637,0.866556,0.011522,0.363528,0.013191,0.818352,0.020518,0.451667,0.062606,0.841489,0.006792,0.401276,0.030892,142.100000,3.604116,31.566667,3.775450,21.933333,2.504249,18.066667,2.504249,3
0.25,0.730349,0.034617,0.860369,0.014870,0.355185,0.018940,0.793216,0.047882,0.465625,0.060098,0.824820,0.030009,0.400937,0.027096,132.775000,16.672544,33.975000,5.404118,21.375000,2.403923,18.625000,2.403923,4
0.25,0.730205,0.031085,0.860718,0.013374,0.349868,0.020378,0.793664,0.043157,0.460500,0.055579,0.825330,0.026949,0.395929,0.026561,134.220000,15.195716,34.380000,5.005670,21.580000,2.223162,18.420000,2.223162,5
0.25,0.728331,0.028762,0.861568,0.012688,0.347255,0.020348,0.790237,0.040123,0.465000,0.053322,0.823910,0.024816,0.396019,0.025628,133.883333,13.872411,35.116667,4.864579,21.400000,2.132874,18.600000,2.132874,6
0.25,0.735593,0.032137,0.864308,0.013612,0.357526,0.031799,0.797800,0.041667,0.469643,0.051233,0.829282,0.026547,0.404407,0.031754,135.957143,13.823353,34.042857,5.254220,21.214286,2.049340,18.785714,2.049340,7
0.25,0.737879,0.030820,0.867624,0.015539,0.363943,0.034658,0.797021,0.039175,0.485625,0.064235,0.830403,0.025106,0.414532,0.040357,135.387500,13.022370,34.112500,4.952927,20.575000,2.569416,19.425000,2.569416,8
0.35,0.733473,0.025314,0.871995,0.013390,0.368657,0.036777,0.783912,0.032770,0.520833,0.065154,0.825279,0.021293,0.431004,0.044022,130.844444,13.684988,35.711111,3.796049,19.166667,2.606172,20.833333,2.606172,9


In [12]:
selected_features = ['Solvent', 'Temp', 'Aliphatic Atom Count', 'Aromatic Atom Count', 'Steric Effect Index', 'Atomic_Polarizability', 'Balaban Index', 'Distance Degree', 'Dreiding Energy', 
                     'Eccentricity', 'Hydrogen Acceptor Count', 'Hydrogen Donor Count', 'MaxZ', 'Sigma Electronegativity']


#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation8-26-24.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 

Val_results.to_csv("JchemExp.csv", mode="a")

Val_results

100%|██████████| 81/81 [00:13<00:00,  5.96it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,1,1,0,0,0,1,0,1,1,1,1
77,0,0,0,0,0,0,0,0,0,0,0
78,0,1,1,1,1,1,1,1,1,1,1
79,0,0,0,0,0,0,0,0,0,0,0
